* DATASET: Apresentar dados que serão utilizados na soluçao.

* ANÁLISE: Planejar e gerar as análises que serão realizadas com os dados propostos em código python.

* INSIGHT: Apresentar resultados (em forma de listagem e/ou gráfico) e gerar insights relacionados a solução proposta. Os insights deverão ser disponibilizados como comentários no arquivo, logo após o resultado. Considere que os insights precisam ser bem compreendidos. Portanto, explique bem seus insights.
 

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from datetime import datetime

In [0]:
# Configurações visuais
#sns.set(style="whitegrid") 
#plt.rcParams["figure.figsize"] = (10,5)

#df = pd.read_csv("dados_mockados.csv")
df = spark.table("bus.bronze.ota_bus_ticket_sales")

# data para datetime
df["date_purchase"] = pd.to_datetime(df["date_purchase"])

In [0]:
# criar colunas derivadas
df["year"] = df["date_purchase"].dt.year
df["month"] = df["date_purchase"].dt.month
df["day_of_week"] = df["date_purchase"].dt.day_name()
print('Amostra')
print(df.head(10),'\n')

In [0]:
# análise explorativa
print("Informações gerais:")
print(df.info())

print("\nEstatísticas descritivas:")
print(df.describe())

In [0]:
# distribuição de gastos
sns.histplot(df["gmv_success"])
plt.xlabel('Valor das compras R$')
plt.ylabel('Frequência de compras')
plt.title("Distribuição dos valores de compras (GMV)") # Gross Merchandise Value - valor total bruto das vendas realizadas em uma plataforma em determinado período de tempo
plt.show()

In [0]:
# destinos mais comuns
top_destinos = df["place_destination_departure"].value_counts().head(10)
sns.barplot(x=top_destinos.values, y=top_destinos.index)
plt.title("Top 10 destinos mais procurados")
plt.show()

In [0]:
# estatísticas por cliente
clientes = df.groupby("fk_contact").agg({
    "gmv_success":"sum",
    "nk_ota_localizer_id":"count",
    "date_purchase":"max"
}).reset_index()
print("Estatísticas por cliente:")
print(clientes.describe())
# análise de clusters

In [0]:
clientes.rename(columns={
    "gmv_success":"total_gasto",
    "nk_ota_localizer_id":"qtd_compras",
    "date_purchase":"ultima_compra"
}, inplace=True)

# recência em dias
max_date = df["date_purchase"].max()
clientes["recencia"] = (max_date - clientes["ultima_compra"]).dt.days

# RFM
rfm = clientes[["recencia","qtd_compras","total_gasto"]].copy()

# normalização simples
rfm_norm = (rfm - rfm.min()) / (rfm.max() - rfm.min())

kmeans = KMeans(n_clusters=4, random_state=42)
clientes["cluster"] = kmeans.fit_predict(rfm_norm)

In [0]:
# visualização
sns.scatterplot(data=clientes, x="qtd_compras", y="total_gasto", hue="cluster", palette="tab10")
plt.title("Clusterização de Clientes (Frequência x Gasto)")
plt.show()

print("\nResumo dos clusters:")
print(clientes.groupby("cluster").agg({
    "qtd_compras":"mean",
    "total_gasto":"mean",
    "recencia":"mean",
    "fk_contact":"count"
}))